# Import my personal printing object

In [5]:
from parameters import ParameterContainer, ParameterColumn, ParameterTable

# Import and name constants

In [6]:
import numpy as np
from scipy import constants
c = constants.physical_constants["speed of light in vacuum"][0] #m/sec by default
e = constants.physical_constants["elementary charge"][0] #m/sec by default
m_e_c2 = constants.physical_constants["electron mass energy equivalent in MeV"][0]*10**-3 #in GeV
r_e = constants.physical_constants["classical electron radius"][0]#in m
h_bar = constants.physical_constants["Planck constant over 2 pi in eV s"][0] #in eV s
alpha = constants.physical_constants["fine-structure constant"][0] #no units

# Name and assign input parameters

In [35]:
column1 = ParameterContainer()
column1.addParameter("E",1.,"GeV")
column1.addParameter("B",1.5,"T")

column2 = ParameterContainer()
column2.addParameter("E",1.,"GeV")
column2.addParameter("B",10.,"T")

column3 = ParameterContainer()
column3.addParameter("E",3.,"GeV")
column3.addParameter("B",10.,"T")

table = ParameterTable()
table.addColumn(ParameterColumn(column1))
table.addColumn(ParameterColumn(column2))
table.addColumn(ParameterColumn(column3))

# Calculate and store derived parameters

In [36]:
for column in table:
    electron_energy = column.header_parameters.getParameter("E").value()
    derived_parameters = ParameterContainer()
    electron_kinetic_energy = derived_parameters.addParameter("electron kinetic energy",(electron_energy - m_e_c2)*10**9,"eV").value()
    derived_parameters.addParameter("electron momentum",np.sqrt(electron_energy**2 - m_e_c2**2)*10**9/c,"eV/c")
    derived_parameters.addParameter("electron gamma",electron_kinetic_energy/(m_e_c2*10**9) + 1)
    if electron_energy == 1.:
        electron_charge = derived_parameters.addParameter("beam charge",10e-12,"C").value()
    elif electron_energy == 3.0:
        electron_charge = derived_parameters.addParameter("beam charge",7e-12,"C").value()
    else:
        raise Exception("You need to pick an electron charge for your pulse.")
    derived_parameters.addParameter("number electrons",electron_charge/e)
    column.associateParameterContainer("Derived",derived_parameters)

table.printTable("Derived")

|  | E = 1.0 GeV,B = 1.5 T | E = 1.0 GeV,B = 10.0 T | E = 3.0 GeV,B = 10.0 T |

| :-: | :-: | :-: | :-: |

| electron kinetic energy (eV) | 999489001.054 | 999489001.054 | 2999489001.05 |

| electron momentum (eV/c) | 3.33564051648 | 3.33564051648 | 10.0069227108 |

| electron gamma | 1956.95119849 | 1956.95119849 | 5870.85359548 |

| beam charge (C) | 1e-11 | 1e-11 | 7e-12 |

| number electrons | 62415091.2588 | 62415091.2588 | 43690563.8812 |

# Ring/turn/magnet parameters

In [97]:
divergence = 0.0005 #Divergence in radians
E_spread = 0.02 # 2% is +/- 1%
vertical_aperature = 0.01 # in m
horizontal_aperature = 0.10 # in m
straight_lengths = 5 # in m; all straight sections of the ring

for column in table:
    electron_energy = column.header_parameters.getParameter("E").value()
    quad_B = column.header_parameters.getParameter("B").value()
    electron_momentum = column.parameter_containers["Derived"].getParameter("electron momentum").value()
    electron_gamma = column.parameter_containers["Derived"].getParameter("electron gamma").value()
    
    ring_parameters = ParameterContainer()
    rho = ring_parameters.addParameter("rho",electron_momentum/quad_B,"m").value()
    perimeter = ring_parameters.addParameter("Perimeter",2*np.pi*rho + straight_lengths,"m").value()
    time_per_revolution = ring_parameters.addParameter("Time per revolution",perimeter/c,"sec").value()
    L_bend = ring_parameters.addParameter("length of bend",np.pi/2.*rho,"m")
    U0 = ring_parameters.addParameter("synchrotron energy loss per turn",4.*np.pi/3.*r_e*electron_gamma**4/rho*m_e_c2*10**6,"keV").value()
    E_photon = ring_parameters.addParameter("energy of sychrotron emitted photon",h_bar*3./2.*c*electron_gamma**3/rho/1000,"keV")
    column.associateParameterContainer("Ring",ring_parameters)
    E_loss_per_turn = ring_parameters.addParameter("synchrotron energy percent loss per turn",U0 / (electron_energy*1e6) * 100,"%").value()
    beam_x = ring_parameters.addParameter("Horizontal beam size",E_spread*rho,"m").value()
    beam_y = ring_parameters.addParameter("Vertical beam size",d*divergence,"m").value()
    ring_parameters.addParameter("Chosen vertical aperature",vertical_aperature,"m")
    ring_parameters.addParameter("Chosen horizontal aperature",horizontal_aperature,"m")
    allowable_e_loss = ring_parameters.addParameter("Allowable E-loss",(horizontal_aperature - beam_x)/(2*rho) * 100,"%").value()
    number_of_turns = ring_parameters.addParameter("Number of turns",allowable_e_loss / E_loss_per_turn).value()
    vertical_angle = ring_parameters.addParameter("Vertical angle",beam_y/(2*perimeter)*1000,"mrad").value()
    horizontal_angle = ring_parameters.addParameter("Horizontal angle",beam_x/(2*perimeter)*1000,"mrad").value()
    beam_size_area = ring_parameters.addParameter("Beam area",np.pi*beam_x*beam_y*1e6,"mm^2").value()
    ring_parameters.addParameter("Emittance",beam_size_area*vertical_angle*horizontal_angle,"mm^2*mrad^2")
    
table.printTable("Ring")


|  | E = 1.0 GeV,B = 1.5 T | E = 1.0 GeV,B = 10.0 T | E = 3.0 GeV,B = 10.0 T |

| :-: | :-: | :-: | :-: |

| rho (m) | 2.22376034432 | 0.333564051648 | 1.00069227108 |

| Perimeter (m) | 18.9722983221 | 7.09584474832 | 11.2875349746 |

| Time per revolution (sec) | 6.32847752365e-08 | 2.36691903314e-08 | 3.76511639083e-08 |

...

| Number of turns | 313.831524017 | 527.502984983 | 16.7440561127 |

| Vertical angle (mrad) | 0.00647504989492 | 0.0173124670302 | 0.0108833840633 |

| Horizontal angle (mrad) | 1.17210909641 | 0.470083638353 | 0.886546330377 |

| Beam area (mm^2) | 34.3289807852 | 5.14934711777 | 15.4480431461 |

| Emittance (mm^2*mrad^2) | 0.260538594088 | 0.0419069702195 | 0.149052362995 |

# Doublet parameters 

In [73]:
L_quad = 0.20 #Length of the doublet quads in m.
max_pole_tip = 7. # Maximum magnetic field at pole tip

for column in table:
    electron_energy = column.header_parameters.getParameter("E").value()
    quad_B = column.header_parameters.getParameter("B").value()
    electron_momentum = column.parameter_containers["Derived"].getParameter("electron momentum").value()
    electron_gamma = column.parameter_containers["Derived"].getParameter("electron gamma").value()
    rho = column.parameter_containers["Ring"].getParameter("rho").value()
    
    doublet_parameters = ParameterContainer()
    doublet_parameters.addParameter("Quad length",L_quad,"m")
    
    if electron_energy == 1.:
        G1 = doublet_parameters.addParameter("Quad gradient 1",24.,"T/m").value()
        G2 = doublet_parameters.addParameter("Quad gradient 2",48.,"T/m").value()    
    elif electron_energy == 3.0:
        G1 = doublet_parameters.addParameter("Quad gradient 1",72.,"T/m").value()
        G2 = doublet_parameters.addParameter("Quad gradient 2",144.,"T/m").value()
    else:
        raise Exception("You need to pick an a_0 for your magnet.")
    K1 = doublet_parameters.addParameter("K1",G1/electron_momentum,"1/m^2").value()
    K2 = doublet_parameters.addParameter("K2",G2/electron_momentum,"1/m^2").value()
    f1 = doublet_parameters.addParameter("f1",electron_momentum/(G1*L_quad),"m").value()
    f2 = doublet_parameters.addParameter("f2",electron_momentum/(G2*L_quad),"m").value()
    d = doublet_parameters.addParameter("Post quad drift",np.sqrt(f2**2 * f1 / (f1-f2)),"m").value()
    quad_spacing = doublet_parameters.addParameter("Quad spacing",f2 * f1 / d,"m").value()
    doublet_parameters.addParameter("Doublet focus",d + quad_spacing,"m")
    doublet_parameters.addParameter("Maximum horizontal aperature", 2*max_pole_tip/G2,"m")
    doublet_parameters.addParameter("Maximum vertical aperature", 2*max_pole_tip/G2,"m")
    
    column.associateParameterContainer("Doublet",doublet_parameters)
    
table.printTable("Doublet")

|  | E = 1.0 GeV,B = 1.5 T | E = 1.0 GeV,B = 10.0 T | E = 3.0 GeV,B = 10.0 T |

| :-: | :-: | :-: | :-: |

| Quad length (m) | 0.2 | 0.2 | 0.2 |

| Quad gradient 1 (T/m) | 24.0 | 24.0 | 72.0 |

| Quad gradient 2 (T/m) | 48.0 | 48.0 | 144.0 |

...

| Post quad drift (m) | 0.491386256001 | 0.491386256001 | 0.491386313028 |

| Quad spacing (m) | 0.491386256001 | 0.491386256001 | 0.491386313028 |

| Doublet focus (m) | 0.982772512002 | 0.982772512002 | 0.982772626056 |

| Maximum horizontal aperature (m) | 0.291666666667 | 0.291666666667 | 0.0972222222222 |

| Maximum vertical aperature (m) | 0.291666666667 | 0.291666666667 | 0.0972222222222 |

# Laser parameters

In [56]:
for column in table:
    electron_energy = column.header_parameters.getParameter("E").value()
    laser_parameters = ParameterContainer()
    laser_wavelength = laser_parameters.addParameter("wavelength",780,"nm").value()
    a_0 = laser_parameters.addParameter("a_0",2.1).value()
    intensity = laser_parameters.addParameter("intensity",a_0**2/(laser_wavelength*1e-3)**2/7.3e-19,"W/cm^2").value()

    column.associateParameterContainer("Laser",laser_parameters)
    
table.printTable("Laser")

|  | E = 1.0 GeV,B = 1.5 T | E = 1.0 GeV,B = 10.0 T | E = 3.0 GeV,B = 10.0 T |

| :-: | :-: | :-: | :-: |

| wavelength (nm) | 780 | 780 | 780 |

| a_0 | 2.1 | 2.1 | 2.1 |

| intensity (W/cm^2) | 9.92948042474e+18 | 9.92948042474e+18 | 9.92948042474e+18 |

# Plasma parameters

In [57]:
for column in table:
    electron_energy = column.header_parameters.getParameter("E").value()
    quad_B = column.header_parameters.getParameter("B").value()
    electron_momentum = column.parameter_containers["Derived"].getParameter("electron momentum").value()
    electron_gamma = column.parameter_containers["Derived"].getParameter("electron gamma").value()
    a_0 = column.parameter_containers["Laser"].getParameter("a_0").value()
    
    plasma_parameters = ParameterContainer()
    n = plasma_parameters.addParameter("plasma density",1.75e17,"cm^-3").value()
    E_max = plasma_parameters.addParameter("Max E field",np.sqrt(n * a_0**2)/1e9,"GV/cm").value()
    
    
    plasma_wavelength = plasma_parameters.addParameter("plasma wavelength",0.1*np.sqrt(1e17/n),"mm").value()
    w_pe = plasma_parameters.addParameter("plasma frequency",2.*np.pi*c/(plasma_wavelength*1e-3),"rad/sec").value()
    R_b = plasma_parameters.addParameter("bubble size",2*np.sqrt(a_0)*c/w_pe,"m").value()
    laser_power = plasma_parameters.addParameter("total laser power",np.pi*R_b**2*intensity*100.**2/1e12,"TW").value()
    L_p = plasma_parameters.addParameter("depletion length",0.5 / a_0 * plasma_wavelength**3 / (laser_wavelength*1e-6)**2/10,"cm").value()
    L_d = plasma_parameters.addParameter("dephasing length",L_p*a_0/10,"cm").value()
    max_energy = plasma_parameters.addParameter("maximum energy possible",E_max*np.min([L_p,L_d]),"GeV").value()
    total_plasma_length = plasma_parameters.addParameter("total plasma length",electron_energy / E_max,"cm").value()

    column.associateParameterContainer("Plasma",plasma_parameters)
    
table.printTable("Plasma")

|  | E = 1.0 GeV,B = 1.5 T | E = 1.0 GeV,B = 10.0 T | E = 3.0 GeV,B = 10.0 T |

| :-: | :-: | :-: | :-: |

| plasma density (cm^-3) | 1.75e+17 | 1.75e+17 | 1.75e+17 |

| Max E field (GV/cm) | 0.878493027861 | 0.878493027861 | 0.878493027861 |

| plasma wavelength (mm) | 0.0755928946018 | 0.0755928946018 | 0.0755928946018 |

...

| total laser power (TW) | 379.278214064 | 379.278214064 | 379.278214064 |

| depletion length (cm) | 16.9045817963 | 16.9045817963 | 16.9045817963 |

| dephasing length (cm) | 3.54996217723 | 3.54996217723 | 3.54996217723 |

| maximum energy possible (GeV) | 3.11861702186 | 3.11861702186 | 3.11861702186 |

| total plasma length (cm) | 1.13831296127 | 1.13831296127 | 3.41493888381 |

# Wiggler parameters

In [102]:
length_of_wiggler = 2. # in m

for column in table:
    electron_energy = column.header_parameters.getParameter("E").value()
    quad_B = column.header_parameters.getParameter("B").value()
    electron_momentum = column.parameter_containers["Derived"].getParameter("electron momentum").value()
    electron_gamma = column.parameter_containers["Derived"].getParameter("electron gamma").value()
    number_electrons = column.parameter_containers["Derived"].getParameter("number electrons").value()
    number_of_turns = column.parameter_containers["Ring"].getParameter("Number of turns").value()
    time_per_revolution = column.parameter_containers["Ring"].getParameter("Time per revolution").value()
    emittance = column.parameter_containers["Ring"].getParameter("Emittance").value()
    
    wiggler_parameters = ParameterContainer()

    if electron_energy == 1.0:
        photon_energy = wiggler_parameters.addParameter("photon energy",0.4,"keV").value()
        wiggler_wavelength = wiggler_parameters.addParameter("wiggler periodicity",1.5e-2,"m").value()
    elif electron_energy == 3.0:
        photon_energy = wiggler_parameters.addParameter("photon energy",10.,"keV").value()
        wiggler_wavelength = wiggler_parameters.addParameter("wiggler periodicity",1.e-1,"m").value()
    else:
        raise Exception("You need to pick an arbitrary photon energy for your electon energy.")
    
    wiggler_rho = wiggler_parameters.addParameter("rho",h_bar*1.5*c*electron_gamma**3/(photon_energy*1e3),"m").value()
    wiggler_B = wiggler_parameters.addParameter("magnetic_field",3.33* electron_momentum / wiggler_rho,"T").value()
    K = wiggler_parameters.addParameter("K",electron_gamma*wiggler_wavelength/(2*np.pi*wiggler_rho),"").value()
    photons_per_unit_length_per_electron = wiggler_parameters.addParameter("photons per m per e",alpha*electron_gamma/wiggler_rho,"1/(m*electron)").value()
    photons_per_unit_length = wiggler_parameters.addParameter("photons per m",photons_per_unit_length_per_electron*number_electrons,"1/m").value()
    energy_loss_per_m_per_electron = wiggler_parameters.addParameter("energy per m",photons_per_unit_length_per_electron*photon_energy,"keV/m").value()
    number_photons_lifetime = wiggler_parameters.addParameter("Average number of photons per sec",photons_per_unit_length*length_of_wiggler*number_of_turns,"photons per sec").value()
    number_photons_spill = wiggler_parameters.addParameter("Average number of photons per spill",photons_per_unit_length*length_of_wiggler / time_per_revolution,"photons per sec").value()
    wiggler_parameters.addParameter("Brilliance per electron lifetime",number_photons_lifetime/emittance,"photons/(mm^2*mrad^2*sec)")
    wiggler_parameters.addParameter("Brilliance per spill",number_photons_spill/emittance,"photons/(mm^2*mrad^2*sec)")
    wiggler_parameters.addParameter("x-ray pulse length",length_of_wiggler/c,"sec")
    wiggler_parameters.addParameter("x-ray train len",number_of_turns*time_per_revolution,"sec")
    
    column.associateParameterContainer("Wiggler",wiggler_parameters)
    
table.printTable("Wiggler")

|  | E = 1.0 GeV,B = 1.5 T | E = 1.0 GeV,B = 10.0 T | E = 3.0 GeV,B = 10.0 T |

| :-: | :-: | :-: | :-: |

| photon energy (keV) | 0.4 | 0.4 | 10.0 |

| wiggler periodicity (m) | 0.015 | 0.015 | 0.1 |

| rho (m) | 5.54571722263 | 5.54571722263 | 5.98937460044 |

...

| Average number of photons per spill (photons per sec) | 5.07934798702e+15 | 1.35807516525e+16 | 1.66006213087e+16 |

| Brilliance per electron lifetime (photons/(mm^2*mrad^2*sec)) | 387196755919.0 | 4.04618622764e+12 | 70214135644.0 |

| Brilliance per spill (photons/(mm^2*mrad^2*sec)) | 1.94955684197e+16 | 3.24069041054e+17 | 1.11374425572e+17 |

| x-ray pulse length (sec) | 6.67128190396e-09 | 6.67128190396e-09 | 6.67128190396e-09 |

| x-ray train len (sec) | 1.98607574595e-05 | 1.24855685519e-05 | 6.30433201191e-07 |